In [ ]:
import os
import sys
import numpy as np
import pickle
import cv2



# # print(type(data))
# # print(data[0]['pov'])
# print(len(data)) # number of steps
# print(len(data[0]['pov'])) # number of envs
# print(np.shape(data[0]['pov'])) # 2,360,640,3

# data1 = data[0]['pov'][0] # 0-step, pov-keys, 0-env_idx
# print(np.shape(data1))


def extract_obs_pov(file_path):
    '''
    
    Extract POV from the observation file
    Args:
        file_path (str): path to the observation file
        POV (dict): dict of np array of POV for each env

    '''

    with open(file_path, 'rb') as f:
        data = pickle.load(f)

    steps = len(data)
    num_env = len(data[0]['pov'])

    # init dict and populate with POV
    POV = {env: [] for env in range(num_env)}
    for step in range(steps):
        for env in range(num_env):
            POV[env].append(data[step]['pov'][env])

    #convert to np array
    for env in POV:
        POV[env] = np.array(POV[env])

    return POV

def gen_vid_from_pov(pov_dict, output_dir, fps=15):
    """
    Generate videos from the POV dictionary.

    Args:
        pov_dict (dict): Dictionary where each key is an environment index, and each value is a NumPy array of shape (steps, height, width, 3).
        output_dir (str): Directory to save the generated videos.
        fps (int): Frames per second for the videos.
    """
    os.makedirs(output_dir, exist_ok=True)

    for env_idx, frames in pov_dict.items():
        if len(frames) == 0:
            print(f"No frames available for environment {env_idx}, skipping video creation.")
            continue

        # Define video file path
        video_path = os.path.join(output_dir, f"env_{env_idx}.mp4")

        # Get frame dimensions
        frame_height, frame_width, _ = frames[0].shape

        # Initialize video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        video_writer = cv2.VideoWriter(video_path, fourcc, fps, (frame_width, frame_height))

        # Write frames to the video
        for frame in frames:
            frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)  # Convert RGB to BGR
            video_writer.write(frame_bgr)

        # Release the video writer
        video_writer.release()
        print(f"Video saved for environment {env_idx} at: {video_path}")

In [2]:
file_path = r"F:\16831_RL\Proj\MC_RL\Playground\001\all_observations.pkl"
output_video_dir = "./001/output_videos"

POV = extract_obs_pov(file_path)

gen_vid_from_pov(POV, output_video_dir, fps=15)

Video saved for environment 0 at: ./001/output_videos\env_0.mp4
Video saved for environment 1 at: ./001/output_videos\env_1.mp4
Video saved for environment 2 at: ./001/output_videos\env_2.mp4
Video saved for environment 3 at: ./001/output_videos\env_3.mp4
